In [1]:
import numpy as np
import pandas as pd
import torch

In [2]:
import os

In [3]:
import pickle

In [4]:
import pylangacq as pla

In [5]:
import random

Load Targets

In [6]:
# y_cookie = torch.load('cookie_data_targets.pt')

In [6]:
y_cookie = torch.load('cookie_target_dict.pt')

In [8]:
len(y_cookie)

552

In [7]:
# y_sentence = torch.load('sentence_data_targets.pt')

In [8]:
y_sentence = torch.load('sentence_target_dict.pt')

In [9]:
len(y_sentence)

240

Load Samples

In [10]:
X_cookie_filenames = torch.load('cookie_fnames_dict.pt')

In [11]:
len(X_cookie_filenames)

552

In [12]:
X_sent_filenames = torch.load('sentence_fnames_dict.pt')

In [13]:
len(X_sent_filenames)

240

In [14]:
num_samples = len(X_cookie_filenames) + len(X_sent_filenames)

In [15]:
num_samples

792

In [16]:
X_cookie_reader = pla.read_chat('./Pitt/*/cookie/*.cha')
X_sentence_reader = pla.read_chat('./Pitt/*/sentence/*.cha')

-----------------------------------

Missing data -- targets

group missing or input error st no mean to replace with and left untouched in target generation

replace with closest analog as per means generated from target generation:

cookie {'Control': 30, 'ProbableAD': 19, 'MCI': 28, 'Memory': 30, 'Vascular': 17, 'PossibleAD': 20, 'Probable': 19, 'Other': 24}

sentence {'ProbableAD': 19, 'MCI': 28, 'Memory': 30, 'Vascular': 17, 'PossibleAD': 21, 'Control': 30, 'Probable': 19, 'Other': 24}

In [19]:
for sent_k in y_sentence:
    if type(y_sentence[sent_k]) == str:
        print(X_sent_filenames[sent_k])
        print(sent_k,y_sentence[sent_k])

In [21]:
target_674 = X_sentence_reader.headers()['/Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Dementia/sentence/236-0.cha']['Participants']['PAR']['group']

In [143]:
target_674

''

In [144]:
group_674

'Dementia'

In [154]:
# Replace Dementia with Possible mean (as per diagnostic codes in Pitt-data.xlxs and accompanying Pitt-readme.pdf)
y_sentence[674] = 21

In [139]:
for coo_kie in y_cookie:
    if type(y_cookie[coo_kie]) == str:
        print(X_cookie_filenames[coo_kie])
        print(coo_kie,y_cookie[coo_kie])

/Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Control/cookie/304-1.cha
219 
/Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Dementia/cookie/585-0.cha
511 possibleAD


In [146]:
target_219 = X_cookie_reader.headers()['/Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Control/cookie/304-1.cha']['Participants']['PAR']['education']
group_219 = X_cookie_reader.headers()['/Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Control/cookie/304-1.cha']['Participants']['PAR']['group']

In [147]:
target_219

''

In [148]:
group_219

''

In [156]:
# Replace '' with Control mean (as per folder and dx codes in .xlxs)
y_cookie[219] = 30

In [149]:
target_511 = X_cookie_reader.headers()['/Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Dementia/cookie/585-0.cha']['Participants']['PAR']['education']
group_511 = X_cookie_reader.headers()['/Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Dementia/cookie/585-0.cha']['Participants']['PAR']['group']

In [150]:
target_511

''

In [151]:
group_511

'possibleAD'

In [158]:
# Replace possibleAD with PossibleAD mean
y_cookie[511] = 20

In [159]:
# confirm all targets filled now 
for sent_k in y_sentence:
    if type(y_sentence[sent_k]) == str:
        print(X_sent_filenames[sent_k])
        print(sent_k,y_sentence[sent_k])


In [160]:
for coo_kie in y_cookie:
    if type(y_cookie[coo_kie]) == str:
        print(X_cookie_filenames[coo_kie])
        print(coo_kie,y_cookie[coo_kie])

In [161]:
# torch.save(y_cookie,'cookie_target_dict.pt')

In [162]:
# torch.save(y_sentence,'sentence_target_dict.pt')

---------------------------------------------------------------

Load Vocab Embeddings

In [17]:
glove_path = './PretrainedWordEmb/'

In [18]:
vocab_emb = pickle.load(open(f'{glove_path}/addb.vocab_emb.glove.42B.300.pkl', 'rb'))

In [19]:
len(vocab_emb)

2188

In [20]:
pos_dict = torch.load('pos_dict.pt')

In [21]:
num_samples

792

In [59]:
# valid_ix = random.sample(range(num_samples),int(.20*num_samples))

In [61]:
# train_ix = [i for i in range(num_samples) if i not in valid_ix]

In [65]:
# torch.save(valid_ix,'valid_ix.pt')

In [67]:
# torch.save(train_ix,'train_ix.pt')

In [23]:
valid_ix = torch.load('valid_ix.pt')

In [24]:
len(valid_ix)

158

In [25]:
train_ix = torch.load('train_ix.pt')

In [26]:
len(train_ix)

634

-------------------------------------------

Group minibatches of similar size 

In [163]:
# file index: sequence length
ix_size_dict = {}
minibatch_ix = train_ix
    
#index, filename
cookie_files = [(i,X_cookie_filenames[i]) for i in minibatch_ix if i in X_cookie_filenames.keys()]
sent_files = [(i,X_sent_filenames[i] )for i in minibatch_ix if i in X_sent_filenames.keys()]


for corpus,data,targetdict in [(cookie_files,X_cookie_reader,y_cookie),(sent_files,X_sentence_reader,y_sentence)]: 
    for file_ix,file in corpus:
#             print(file_ix, file)
#             print('Words',[tokensraw for utterance in data.tagged_sents(participant='PAR',by_files=True)[file] for (tokensraw,pos,tokenstem,dependency) in utterance])
#             print('Words_len',len([tokensraw for utterance in data.tagged_sents(participant='PAR',by_files=True)[file] for (tokensraw,pos,tokenstem,dependency) in utterance]))
        embedding = [(vocab_emb[token],torch.zeros(len(pos_dict),dtype=torch.float64),pos_dict[pos]) for (token,pos) in zip([tokensraw for utterance in data.tagged_sents(participant='PAR',by_files=True)[file] for (tokensraw,pos,tokenstem,dependency) in utterance], [pos for utterance in data.tagged_sents(participant='PAR',by_files=True)[file] for (tokensraw,pos,tokenstem,dependency) in utterance])]
        ix_size_dict[file_ix] = len(embedding)
#         target = targetdict[file_ix]

In [171]:
# sequence length: [file indices]
size_ix_dict = {}
for k,v in ix_size_dict.items():
    if v not in size_ix_dict.keys():
        size_ix_dict[v] = [k]
    else:
        size_ix_dict[v].append(k)

In [291]:
# set of sequence-lengths
len_sents = sorted(list(size_ix_dict.keys()),reverse=True)

In [220]:
batches = []
while len(len_sents) > 0:
    minibsize = 0
    batchset = []
    while minibsize < 25:
        batchset.extend(size_ix_dict[len_sents[0]])
        minibsize = len(batchset)
        len_sents.pop(0)
    batches.append(batchset)

IndexError: list index out of range

In [223]:
# append final batchset
batches.append(batchset)

In [232]:
# confirm all train files accounted for 
sum([1 for blist in batches for bitem in blist])

634

In [233]:
len(train_ix)

634

-------------------------------------------

In [27]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

In [246]:
#pad with zeros -- maybe
# end with eos tensor, tag

def get_minibatch(batchsize=1, ix=None,pad=True):
    
    
    def num_vectorize(targets):
        vectors = []
        for i in targets:
            v = torch.zeros([30])
            # up until MMSE index...greater than index i 
            v[:i] = 1
            vectors.append(v)
        return torch.stack(vectors).double()
    
    def pad_minibatch(minib):
    
        batchsize = len(minib)

        seq_lens = [len(emb) for emb in minib]
        max_len = max(seq_lens)

        # input of shape (seq_len, batch, input_size): tensor containing the features of the input sequence
        # https://pytorch.org/docs/stable/generated/torch.nn.GRU.html
    #     seq_tensor = torch.zeros((batchsize, max_len, EMBEDDING_SIZE)).double()
        seq_tensor = torch.zeros((max_len, batchsize, EMBEDDING_SIZE)).double()


        for i, (seq,length) in enumerate( zip(minib,seq_lens) ):
            for wi,word in enumerate(seq):
    #             seq_tensor[i,wi] = word
                seq_tensor[wi,i] = word
        # mod to sort at the start?
        seq_tensor = nn.utils.rnn.pack_padded_sequence(seq_tensor,lengths=seq_lens,enforce_sorted=False)
        return seq_tensor
    
    
    minibatch_ix = random.sample(range(num_samples),batchsize) if ix is None else ix 
    
    #index, filename
    cookie_files = [(i,X_cookie_filenames[i]) for i in minibatch_ix if i in X_cookie_filenames.keys()]
    sent_files = [(i,X_sent_filenames[i] )for i in minibatch_ix if i in X_sent_filenames.keys()]
    
    embeddings = []
    targets = []

    for corpus,data,targetdict in [(cookie_files,X_cookie_reader,y_cookie),(sent_files,X_sentence_reader,y_sentence)]: 
        for file_ix,file in corpus:
#             print(file_ix, file)
#             print('Words',[tokensraw for utterance in data.tagged_sents(participant='PAR',by_files=True)[file] for (tokensraw,pos,tokenstem,dependency) in utterance])
#             print('Words_len',len([tokensraw for utterance in data.tagged_sents(participant='PAR',by_files=True)[file] for (tokensraw,pos,tokenstem,dependency) in utterance]))
            embedding = [(vocab_emb[token],torch.zeros(len(pos_dict),dtype=torch.float64),pos_dict[pos]) for (token,pos) in zip([tokensraw for utterance in data.tagged_sents(participant='PAR',by_files=True)[file] for (tokensraw,pos,tokenstem,dependency) in utterance], [pos for utterance in data.tagged_sents(participant='PAR',by_files=True)[file] for (tokensraw,pos,tokenstem,dependency) in utterance])]
            target = targetdict[file_ix]
            
            for tkn in embedding:

                tkn[1][tkn[2]] = 1

#                 embeddings.append(torch.cat((tkn[0],tkn[1])))
            
            embeddings.append([torch.cat((tkn[0],tkn[1])) for tkn in embedding])


#             embeddings.append(embedding)
            targets.append(target)
#     print(targets)       
#     return embeddings, torch.tensor(targets), minibatch_ix
#     return embeddings, torch.tensor(targets)
    if pad: embeddings = pad_minibatch(embeddings)
# return target vals for accuracy 
    return embeddings, num_vectorize(targets), torch.tensor(targets)

In [308]:
# https://towardsdatascience.com/simple-trick-to-train-an-ordinal-regression-with-any-classifier-6911183d2a3c
# transform proba dist prediction to MMSE target
def out_to_score_proba(yhat_vector):
    # mod to 31 for ix 0-30
    proba_vector = torch.zeros([31])
    # no zero-score
    proba_vector[0] = 0. 
# P(MMSE=i) = P(MMSE>i-1) - P(MMSE>i)
    for i in range(1,len(yhat_vector)):
        proba_vector[i] = yhat_vector[i-1] - yhat_vector[i]
        
    proba_vector[-1] = yhat_vector[-1] 
    
#     print(proba_vector)
    return torch.argmax(proba_vector)


In [92]:
# not on y but on targets as mmse scores
def accuracy_distansum(yhat_tensor,y_tensor):
    return (torch.abs(y - yhat).sum())

In [91]:
# raw accurate prediction count
def accuracy(yhat_tensor,y_tensor):
#     return torch.stack([ya==yb for (ya,yb) in zip(yhat_tensor,y_tensor)]).sum()
    return ((yhat_tensor == y_tensor).sum())

------------------------------

MODEL 

2-layer rnn: gru units

In [33]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [35]:
HIDDEN_SIZE = 128
# EMBEDDING_SIZE = 371
EMBEDDING_SIZE = len(vocab_emb['I']) + len(pos_dict)

In [268]:
# max([len(emb) for emb in e])

546

In [36]:
# change if aadd in EOS, pos to 547
MAX_LENGTH = 546

In [37]:
VOCAB_SIZE = len(vocab_emb)

In [38]:
BATCH_SIZE = 32

In [107]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size=MAX_LENGTH, emb_size=EMBEDDING_SIZE, hidden_size=HIDDEN_SIZE):
        super(EncoderRNN, self).__init__()
        
        self.emb_size = emb_size
        self.hidden_size = hidden_size

        self.gru = nn.GRU(input_size=self.emb_size,hidden_size=self.hidden_size,num_layers=2).double()
#         # each output node Oi of our neural network
# uses a standard sigmoid function 1
# 1+e−zi
# , without including
# the outputs from other nodes, as shown in Figure 1. Output
# node Oi
# is used to estimate the probability oi
# that a data
# point belongs to category i independently, without subjecting
# to normalization as traditional neural networks do. Thus,
# for a data point x of category k, the target vector is
# (1, , 1, .., 1, 0, 0, 0), in which the first k elements is 1 and
# others 0
# http://orca.st.usm.edu/~zwang/files/rank.pdf A Neural Network Approach to Ordinal Regression
    
        self.fc = nn.Linear(self.hidden_size*2,30).double()
        
        self.activation = nn.Sigmoid()

    def forward(self, input, hidden):

        embedded = input.double()

        gru_out, _ = self.gru(embedded,hidden)
        
        hid_out = torch.cat((_[-2,:,:], _[-1,:,:]), dim = 1)

        out = self.fc(hid_out)
#         print(out.size())

        out - self.activation(out)

        return out, _
#         return torch.nn.utils.rnn.pad_packed_sequence(out), _

    def initHidden(self,batch_size=BATCH_SIZE):
        # h_0 of shape (num_layers * num_directions, batch, hidden_size): tensor containing the initial hidden state for each element in the batch. If the LSTM is bidirectional, num_directions should be 2
        return torch.zeros(2, batch_size, self.hidden_size, device=device).double()
        


In [108]:
# The
# cost function for a data point x can be relative entropy
# or square error between the target vector and the output
# vector
# loss = nn.BCELoss()
loss_func = nn.MSELoss()

In [247]:
encoder = EncoderRNN()

In [248]:
optimizer = torch.optim.Adam(encoder.parameters(), lr=1e-2)

In [321]:
encoder.train()
# _hidden = encoder.initHidden()
# Group batches/paddings by relative same-size

accuracy_floor = .6
distance_floor = 3.6

for epoch in range(25,27):
    
    # modify for ordering -- benefit here with shuffling at ea epoch
#     random.shuffle(train_ix)
    random.shuffle(batches)
    
    print('Epoch',epoch)
    print('===========')
    step = 0
    
    distance = 0
    accurate = 0
    
    # in future, mix up epochs 
    for i in range(len(batches)):
        batch = batches[i]
#     for i in range(19):
        
        step += 1
        optimizer.zero_grad()
        
#         x,y,targets = get_minibatch(batchsize=BATCH_SIZE)
        x,y,targets = get_minibatch(batchsize=len(batch))
#         x = pad_minibatch(x)
        
#         y = torch.stack(y).double()
        
#         _hidden = encoder.initHidden(batch_size=BATCH_SIZE)
        _hidden = encoder.initHidden(batch_size=len(batch))

        yhat, _hidden = encoder(x, _hidden)

        loss = loss_func(yhat,y)

        loss.backward()
#         loss.backward(retain_graph=True)
        
        optimizer.step()

       
        print('step',step)
        print('loss',loss)

        yhat_tensor = torch.tensor([out_to_score_proba(hat) for hat in yhat])
        
        _stepdistance = accuracy_distansum(yhat_tensor=yhat_tensor,y_tensor=targets)
        distance += _stepdistance
        
        _stepaccurate = accuracy(yhat_tensor=yhat_tensor,y_tensor=targets)
        accurate += _stepaccurate
        
        
        print('step_accuracy_distance',_stepdistance)
        print('step_accurate',_stepaccurate)
    
    print('Accurate_total',accurate)
    print('Distance_total',distance)
    
    epoch_accuracy = accurate.item()/(len(train_ix))
    # may be a bug in distance..
    epoch_distance_avg = distance.item()/(len(train_ix))
    
    print('Epoch_accuracy',epoch_accuracy)
    print('Epoch_distance_avg',epoch_distance_avg)
    
    if epoch_accuracy > accuracy_floor or epoch_distance_avg < distance_floor:
        
        print('Saving..')
        torch.save(encoder.state_dict(),'encoder_accuracy_{:.3f}_avgdistance_{:3f}.pt'.format(epoch_accuracy, epoch_distance_avg))
        
        if epoch_accuracy > accuracy_floor: accuracy_floor = epoch_accuracy
        if epoch_distance_avg > distance_floor: distance_floor = epoch_distance_avg
        
    
        
    
    
    

Epoch 25
torch.Size([26, 30])
step 1
loss tensor(0.0036, dtype=torch.float64, grad_fn=<MseLossBackward>)
step_accuracy_distance tensor(31.8367, dtype=torch.float64, grad_fn=<SumBackward0>)
step_accurate tensor(26)
torch.Size([13, 30])
step 2
loss tensor(0.0037, dtype=torch.float64, grad_fn=<MseLossBackward>)
step_accuracy_distance tensor(15.9313, dtype=torch.float64, grad_fn=<SumBackward0>)
step_accurate tensor(13)
torch.Size([26, 30])
step 3
loss tensor(0.0036, dtype=torch.float64, grad_fn=<MseLossBackward>)
step_accuracy_distance tensor(32.3432, dtype=torch.float64, grad_fn=<SumBackward0>)
step_accurate tensor(26)
torch.Size([32, 30])
step 4
loss tensor(0.0035, dtype=torch.float64, grad_fn=<MseLossBackward>)
step_accuracy_distance tensor(37.2153, dtype=torch.float64, grad_fn=<SumBackward0>)
step_accurate tensor(32)
torch.Size([25, 30])
step 5
loss tensor(0.0031, dtype=torch.float64, grad_fn=<MseLossBackward>)
step_accuracy_distance tensor(27.8402, dtype=torch.float64, grad_fn=<SumBac

In [255]:

encoder.eval()

x_valid,y_valid,targets_valid = get_minibatch(ix=valid_ix)

# x_valid = pad_minibatch(x_valid)

# y_valid = torch.stack(y_valid).double()

_hidden = encoder.initHidden(batch_size=len(valid_ix))

yhat_valid, _hidden = encoder(x_valid, _hidden)
estimates = torch.tensor([out_to_score_proba(hat) for hat in yhat_valid])


torch.Size([158, 30])


In [264]:
targets_valid

tensor([30, 18, 29, 28, 26, 27, 23, 30, 19, 17, 20, 17, 25, 30, 23, 29, 27, 30,
        19, 16, 30,  7, 28, 28, 28, 28, 25, 12, 30, 18, 30, 25, 15, 18, 14, 26,
        27, 30, 20, 17, 30, 26, 19, 27, 17, 20, 20, 29, 27, 29, 30, 30, 30, 20,
        22, 29, 19, 30, 30, 30, 30, 22, 17, 27, 27, 30, 30, 23, 30, 30, 19, 25,
        30, 19, 30, 30, 30,  8, 30, 22, 30, 30, 28, 16, 19, 22, 24, 28, 16, 30,
        30, 19, 17, 27, 19, 30, 30, 21, 28, 19, 29, 19, 12, 28, 20, 23, 13, 20,
        23, 27, 23, 19, 20, 19, 19, 19, 27, 16, 10, 20, 17, 24, 24, 23, 16, 20,
        18, 17, 19, 28, 29, 20, 27, 20, 23, 16, 20, 23, 27, 22, 19, 20, 20, 19,
        19, 28, 28, 28, 26, 19, 24, 22, 20, 18, 23, 27, 25, 25])

In [312]:
estimates

tensor([30, 18, 29, 30, 26, 27, 23, 30, 19, 17, 20, 17, 25, 30, 23, 29, 27, 30,
        19, 16, 30,  7, 28, 28, 28, 28, 25, 12, 30, 18, 30, 25, 15, 18, 14, 26,
        27, 30, 20, 17, 30, 26, 19, 27, 17, 20, 20, 29, 27, 29, 30, 30, 30, 20,
        22, 29, 19, 30, 30, 30, 30, 22, 17, 27, 27, 30, 30, 23, 30, 30, 19, 25,
        30, 19, 30, 30, 30,  8, 30, 22, 30, 30, 28, 16, 19, 22, 24, 28, 16, 30,
        30, 19, 17, 27, 19, 30, 30, 21, 28, 19, 29, 19, 12, 28, 20, 23, 13, 20,
        23, 27, 23, 19, 20, 19, 19, 19, 27, 16, 10, 20, 17, 24, 24, 23, 16, 20,
        18, 17, 19, 28, 29, 20, 27, 20, 23, 16, 20, 23, 27, 22, 19, 20, 20, 19,
        19, 28, 28, 28, 26, 19, 24, 22, 20, 18, 23, 27, 25, 25])

In [303]:
# Export for heatmap visualization
# pd.Series(targets_valid).to_csv('targets_valid.csv')

In [304]:
# pd.Series(estimates).to_csv('estimates.csv')

In [256]:
# what is WRONG with this metric
accuracy_distansum(targets_valid,estimates)

tensor(14.6159, dtype=torch.float64, grad_fn=<SumBackward0>)

In [299]:
14.6159/158

0.09250569620253164

In [257]:
accuracy(targets_valid,estimates)

tensor(157)

In [258]:
157/158

0.9936708860759493

In [259]:
np.where(targets_valid != estimates)

(array([3]),)

In [305]:
encoder.eval()

x_train,y_train,targets_train = get_minibatch(ix=train_ix)

# x_valid = pad_minibatch(x_valid)

# y_valid = torch.stack(y_valid).double()

_hidden = encoder.initHidden(batch_size=len(train_ix))
yhat_train, _hidden = encoder(x_train, _hidden)
estimates_train = torch.tensor([out_to_score_proba(hat) for hat in yhat_train])

torch.Size([634, 30])


In [306]:
targets_train

tensor([28, 30, 12, 17, 30, 29, 20, 29, 15, 30, 29, 29, 29, 29, 30, 30, 28,  3,
        28, 27, 30, 19, 27, 30, 10, 30, 30, 17, 23, 30, 30, 19, 30, 30, 30, 30,
        30, 28, 30, 19, 29, 30, 30, 29, 30, 11, 27, 21, 19, 29, 30, 28, 30, 12,
        30, 13, 22, 21, 29, 29, 17, 10, 23, 13, 18, 11, 28, 30, 18,  3, 29, 27,
        30, 21, 20, 25, 30, 28, 30, 17, 24, 26, 20, 27, 30, 30, 19, 28, 24, 25,
        20, 24, 30, 29, 16, 30, 30, 30, 30, 30, 19, 28, 15, 30, 30, 26, 30, 29,
        30, 30, 30, 26, 30, 19, 26, 23, 18, 19, 13, 23, 12, 25, 30, 30, 10, 24,
        30, 30, 28, 28, 15, 30, 30, 30, 24, 29, 28, 30, 28, 30, 23, 23, 20, 30,
        29,  8, 20, 30, 19, 20, 25, 29, 20, 30, 30, 28, 30, 27, 29, 28, 22, 28,
        25, 23, 29, 20, 23, 16, 18, 23, 19, 25, 30, 30, 17, 30, 30, 20, 10, 30,
        18, 22, 24, 15, 29, 30, 19, 18, 28, 30, 30, 29, 30, 30, 30, 22, 29, 30,
        29, 30, 26, 29, 15, 16, 30, 30, 13, 24, 30, 29, 23, 28, 13, 30, 28, 11,
        29, 18, 18, 17, 27, 16, 12, 13, 

In [314]:
estimates_train_bugfix = torch.tensor([out_to_score_proba(hat) for hat in yhat_train])

In [315]:
estimates_train_bugfix

tensor([28, 30, 12, 17, 30, 29, 20, 29, 15, 30, 29, 29, 29, 29, 30, 30, 28,  3,
        28, 27, 30, 19, 27, 30, 10, 30, 30, 17, 23, 30, 30, 19, 30, 30, 30, 30,
        30, 28, 30, 19, 29, 30, 30, 29, 30, 11, 27, 21, 19, 29, 30, 28, 30, 12,
        30, 13, 22, 21, 29, 29, 17, 10, 23, 13, 18, 11, 28, 30, 18,  3, 29, 27,
        30, 21, 20, 25, 30, 28, 30, 17, 24, 26, 20, 27, 30, 30, 19, 28, 24, 25,
        20, 24, 30, 29, 16, 30, 30, 30, 30, 30, 19, 28, 15, 30, 30, 26, 30, 29,
        30, 30, 30, 26, 30, 19, 26, 23, 18, 19, 13, 23, 12, 25, 30, 30, 10, 24,
        30, 30, 28, 28, 15, 30, 30, 30, 24, 29, 28, 30, 28, 30, 23, 23, 20, 30,
        29,  8, 20, 30, 19, 20, 25, 29, 20, 30, 30, 28, 30, 27, 29, 28, 22, 28,
        25, 23, 29, 20, 23, 16, 18, 23, 19, 25, 30, 30, 17, 30, 30, 20, 10, 30,
        18, 22, 24, 15, 29, 30, 19, 18, 28, 30, 30, 29, 30, 30, 30, 22, 29, 30,
        29, 30, 26, 29, 15, 16, 30, 30, 13, 24, 30, 29, 23, 28, 13, 30, 28, 11,
        29, 18, 18, 17, 27, 16, 12, 13, 

In [316]:
accuracy(targets_train,estimates_train_bugfix)

tensor(632)

In [317]:
632/len(train_ix)

0.9968454258675079

In [318]:
accuracy_distansum(targets_train,estimates_train_bugfix)

tensor(14.6159, dtype=torch.float64, grad_fn=<SumBackward0>)

In [319]:
14.6159/len(train_ix)

0.02305347003154574